In [142]:
import pandas as pd
import numpy as np
import random

In [143]:
df = pd.read_csv('./data/Football teams.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   team          98 non-null     object 
 1   tournament    98 non-null     object 
 2   goals         98 non-null     int64  
 3   shots_pg      98 non-null     float64
 4   yellow_cards  98 non-null     int64  
 5   red_cards     98 non-null     int64  
 6   possession%   98 non-null     float64
 7   pass%         98 non-null     float64
 8   aerialswon    98 non-null     float64
 9   rating        98 non-null     float64
dtypes: float64(5), int64(3), object(2)
memory usage: 7.8+ KB
None


,team,tournament,goals,shots_pg,yellow_cards,red_cards,possession%,pass%,aerialswon,rating
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01
1,Bayern Munich,Bundesliga,99,17.1,44,3,58.1,85.5,12.9,6.95
2,Paris Saint-Germain,Ligue 1,86,15.0,73,7,60.1,89.5,9.5,6.88
3,Barcelona,LaLiga,85,15.3,68,2,62.4,89.7,10.6,6.87
4,Real Madrid,LaLiga,67,14.4,57,2,57.7,87.7,11.8,6.86


In [144]:
df.tournament.unique()

array(['Premier League', 'Bundesliga', 'Ligue 1', 'LaLiga', 'Serie A'],
      dtype=object)

In [145]:
df.tournament.value_counts()

Premier League    20
Ligue 1           20
LaLiga            20
Serie A           20
Bundesliga        18
Name: tournament, dtype: int64

In [146]:
df[['team', 'tournament']].value_counts()

team                     tournament    
AC Milan                 Serie A           1
Rennes                   Ligue 1           1
Real Valladolid          LaLiga            1
Real Sociedad            LaLiga            1
Real Madrid              LaLiga            1
                                          ..
Elche                    LaLiga            1
Eintracht Frankfurt      Bundesliga        1
Eibar                    LaLiga            1
Dijon                    Ligue 1           1
Wolverhampton Wanderers  Premier League    1
Length: 98, dtype: int64

In [147]:
leagues_dict = {'Premier League': 'England', 'Bundesliga': 'Germany', 'Ligue 1': 'France', 'Serie A': 'Italy', 
                'LaLiga': 'Spain'}
df['country'] = df['tournament'].map(leagues_dict)
df.head()

,team,tournament,goals,shots_pg,yellow_cards,red_cards,possession%,pass%,aerialswon,rating,country
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01,England
1,Bayern Munich,Bundesliga,99,17.1,44,3,58.1,85.5,12.9,6.95,Germany
2,Paris Saint-Germain,Ligue 1,86,15.0,73,7,60.1,89.5,9.5,6.88,France
3,Barcelona,LaLiga,85,15.3,68,2,62.4,89.7,10.6,6.87,Spain
4,Real Madrid,LaLiga,67,14.4,57,2,57.7,87.7,11.8,6.86,Spain


In [148]:
df = df[['team', 'tournament', 'country']]
df.head()

,team,tournament,country
0,Manchester City,Premier League,England
1,Bayern Munich,Bundesliga,Germany
2,Paris Saint-Germain,Ligue 1,France
3,Barcelona,LaLiga,Spain
4,Real Madrid,LaLiga,Spain


In [149]:
df_cross = df.merge(df, on='country')
df_cross = df_cross.loc[df_cross.team_x!=df_cross.team_y, :]
df_cross.drop('tournament_y', inplace=True, axis=1)
df_cross.rename(columns={'team_x': 'home_team', 'team_y': 'away_team', 'tournament_x': 'tournament'}, inplace=True)
df_cross = df_cross[['home_team', 'away_team', 'tournament', 'country']]
df_cross

,home_team,away_team,tournament,country
1,Manchester City,Manchester United,Premier League,England
2,Manchester City,Aston Villa,Premier League,England
3,Manchester City,Chelsea,Premier League,England
4,Manchester City,Liverpool,Premier League,England
5,Manchester City,Tottenham,Premier League,England
...,...,...,...,...
1918,Benevento,Genoa,Serie A,Italy
1919,Benevento,Fiorentina,Serie A,Italy
1920,Benevento,Spezia,Serie A,Italy
1921,Benevento,Parma Calcio 1913,Serie A,Italy


In [150]:
df_cross = df_cross.sample(frac=1).sort_values(by='tournament').reset_index(drop=True)
df_cross

,home_team,away_team,tournament,country
0,FC Koln,Bayern Munich,Bundesliga,Germany
1,Augsburg,Mainz 05,Bundesliga,Germany
2,Borussia Dortmund,Augsburg,Bundesliga,Germany
3,Borussia Dortmund,Hoffenheim,Bundesliga,Germany
4,Augsburg,Bayern Munich,Bundesliga,Germany
...,...,...,...,...
1821,Roma,Fiorentina,Serie A,Italy
1822,Torino,AC Milan,Serie A,Italy
1823,AC Milan,Udinese,Serie A,Italy
1824,Roma,Udinese,Serie A,Italy


In [151]:
df_cross['tournament'].value_counts()

LaLiga            380
Ligue 1           380
Premier League    380
Serie A           380
Bundesliga        306
Name: tournament, dtype: int64

In [152]:
minutes_df = pd.DataFrame(np.arange(0, 91, 1), columns=['minute'])
minutes_df.head()

,minute
0,0
1,1
2,2
3,3
4,4


In [153]:
# df_cross.merge(minutes_df, how='cross')
years = [2022, 2023]

In [154]:
df_cross.head()

,home_team,away_team,tournament,country
0,FC Koln,Bayern Munich,Bundesliga,Germany
1,Augsburg,Mainz 05,Bundesliga,Germany
2,Borussia Dortmund,Augsburg,Bundesliga,Germany
3,Borussia Dortmund,Hoffenheim,Bundesliga,Germany
4,Augsburg,Bayern Munich,Bundesliga,Germany


In [155]:
year = pd.DataFrame(np.arange(2022, 2024, 1), columns=['year'])
month = pd.DataFrame(np.arange(1, 13, 1), columns=['month'])
day = pd.DataFrame(np.arange(1, 31, 1), columns=['day'])
dates_df = year.merge(month, how='cross').merge(day, how='cross')
dates_df

,year,month,day
0,2022,1,1
1,2022,1,2
2,2022,1,3
3,2022,1,4
4,2022,1,5
...,...,...,...
715,2023,12,26
716,2023,12,27
717,2023,12,28
718,2023,12,29


In [156]:
df_cross = df_cross.merge(dates_df, how='cross').groupby(['home_team', 'away_team', 'tournament', 'country', 'year']).apply(lambda x: x.sample(1)).reset_index(drop=True)
df_cross.head()

,home_team,away_team,tournament,country,year,month,day
0,AC Milan,Atalanta,Serie A,Italy,2022,12,2
1,AC Milan,Atalanta,Serie A,Italy,2023,12,20
2,AC Milan,Benevento,Serie A,Italy,2022,3,25
3,AC Milan,Benevento,Serie A,Italy,2023,10,8
4,AC Milan,Bologna,Serie A,Italy,2022,5,10


In [157]:
df_cross = df_cross.merge(minutes_df, how='cross')
df_cross

,home_team,away_team,tournament,country,year,month,day,minute
0,AC Milan,Atalanta,Serie A,Italy,2022,12,2,0
1,AC Milan,Atalanta,Serie A,Italy,2022,12,2,1
2,AC Milan,Atalanta,Serie A,Italy,2022,12,2,2
3,AC Milan,Atalanta,Serie A,Italy,2022,12,2,3
4,AC Milan,Atalanta,Serie A,Italy,2022,12,2,4
...,...,...,...,...,...,...,...,...
332327,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,86
332328,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,87
332329,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,88
332330,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,89


In [158]:
df_cross['home_score'] = np.random.choice([0, 1], size=len(df_cross), p=[0.99, 0.01])
df_cross['away_score'] = np.random.choice([0, 1], size=len(df_cross), p=[0.99, 0.01])
df_cross.loc[df_cross.minute==0, 'home_score'] = 0
df_cross.loc[df_cross.minute==0, 'away_score'] = 0
df_cross

,home_team,away_team,tournament,country,year,month,day,minute,home_score,away_score
0,AC Milan,Atalanta,Serie A,Italy,2022,12,2,0,0,0
1,AC Milan,Atalanta,Serie A,Italy,2022,12,2,1,0,0
2,AC Milan,Atalanta,Serie A,Italy,2022,12,2,2,0,0
3,AC Milan,Atalanta,Serie A,Italy,2022,12,2,3,0,1
4,AC Milan,Atalanta,Serie A,Italy,2022,12,2,4,0,0
...,...,...,...,...,...,...,...,...,...,...
332327,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,86,0,0
332328,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,87,0,0
332329,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,88,0,0
332330,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,89,0,0


In [159]:
df_cross['home_score'] = df_cross.groupby(list(df_cross.columns.drop(['minute', 'home_score', 
                                                                 'away_score'])))['home_score'].cumsum()
df_cross['away_score'] = df_cross.groupby(list(df_cross.columns.drop(['minute', 'home_score', 
                                                                 'away_score'])))['away_score'].cumsum()
df_cross

,home_team,away_team,tournament,country,year,month,day,minute,home_score,away_score
0,AC Milan,Atalanta,Serie A,Italy,2022,12,2,0,0,0
1,AC Milan,Atalanta,Serie A,Italy,2022,12,2,1,0,0
2,AC Milan,Atalanta,Serie A,Italy,2022,12,2,2,0,0
3,AC Milan,Atalanta,Serie A,Italy,2022,12,2,3,0,1
4,AC Milan,Atalanta,Serie A,Italy,2022,12,2,4,0,1
...,...,...,...,...,...,...,...,...,...,...
332327,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,86,0,1
332328,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,87,0,1
332329,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,88,0,1
332330,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,89,0,1


In [161]:
df_cross.tail(90)

,home_team,away_team,tournament,country,year,month,day,minute,home_score,away_score
332242,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,1,0,0
332243,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,2,0,0
332244,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,3,0,0
332245,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,4,0,0
332246,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,5,0,0
...,...,...,...,...,...,...,...,...,...,...
332327,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,86,0,1
332328,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,87,0,1
332329,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,88,0,1
332330,Wolverhampton Wanderers,West Ham,Premier League,England,2023,8,30,89,0,1


In [ ]:
df_cross.to_csv('.')